In [1]:
'''
#This program for monitering ant-miner (L3 and s19j-pro)
get asic status from  asic and save to InfluxDB every minutes.
And visualize data with Grafana Dashboard
'''

In [ ]:
#!/usr/bin/python
import socket
import json
import re
import os
import sys, getopt, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

In [ ]:
url='http://influxDB:8086'
token = "influxDB access token"
org = "myOrg"
bucket = "myBucket"

In [ ]:
iplist=["192.168.0.1","192.168.0.2","192.168.0.3","192.168.0.4","192.168.0.5"]

In [ ]:
api_version = '1'

In [ ]:
class CgminerStats(object):
    def __init__(self, host='localhost', port=4028):
        self.data = {}
        self.host = host
        self.port = port
        self.measurement='miner_test'
        self.model='L3++'
        self.stat_data=[]
        self.stats ={}
        self.cmd_result ={}
        self.flat_stat={}
        self.stat_default={}
        self.connected=False


    def resetStat(self):
        self.stat_data = [{"measurement":self.measurement,
                           "tags":{'model':self.model,
                                   'ip':self.host},                           
                          }
        ] 
        self.stat_data[0]['fields'] = self.stat_default.copy()

    def setStatData(self):
        self.stat_data = [{"measurement":self.measurement,
                           "tags":{'model':self.model,
                                   'ip':self.host},
                           "fields":{'temp1':self.stats["STATS"][1]["temp1"]  ,
                                     'temp2':self.stats["STATS"][1]["temp2"]  ,
                                     'temp3':self.stats["STATS"][1]["temp3"]  ,
                                     'temp2_1':self.stats["STATS"][1]["temp2_1"]  ,
                                     'temp2_2':self.stats["STATS"][1]["temp2_2"]  ,
                                     'temp2_3':self.stats["STATS"][1]["temp2_3"]  ,
                                     'hashRate_av':self.stats["STATS"][1]["GHS av"]
                                    }
                          }
        ]

    def addField(self,key_,value_,reset_=False):
        str_field=['STATUS','Msg','Description','CGMiner','Miner','CompileTime','Type','chain_acs1','chain_acs2','chain_acs3','chain_acs4','GHS 5s']
        chip_x = ['chain_acs1','chain_acs2','chain_acs3','chain_acs4']
        temp_x=['temp_pcb1','temp_pcb2','temp_pcb3','temp_pcb4','temp_chip1','temp_chip2','temp_chip3','temp_chip4','temp_pic1','temp_pic2','temp_pic3','temp_pic4']

        if(key_ in str_field):
            self.flat_stat[str(key_).replace(' ','_')]=str(value_)
        else:
            self.flat_stat[str(key_).replace(' ','_')]=value_
            
        
        if( key_ in chip_x):
            self.flat_stat[str(key_)+'x']=str(value_).count('x')
            
        if( key_ in temp_x):
            i=0
            for t in value_.split('-'):
                i = i+1
                self.flat_stat[str(key_)+'_'+str(i)]=str(t)
        
        if(reset_):
            if(type(value_)== str ):
                self.flat_stat[str(key_).replace(' ','_')]=''
            elif (type(value_)== int ):
                self.flat_stat[str(key_).replace(' ','_')] = 0;
            elif(type(value_)== float ):
                self.flat_stat[str(key_).replace(' ','_')] = 0.0
            else :
                self.flat_stat[str(key_).replace(' ','_')]=''
                
        return ''
    
    def flatJson(self,data,reset_=False):
        #print('flatJson' )
        
        for key,value in data.items():
            #print (str(key)+'->'+str(value))
            if isinstance(value, dict):
                self.flatJson(value,reset_)
            elif isinstance(value, list):
                for val in value:
                    if isinstance(val, str):
                        #print (str(key)+'->'+str(value))
                        pass
                    elif isinstance(val, list):
                        pass
                    else:
                        self.flatJson(val,reset_)
            else:

                #print ('\'' + str(key)+'\':'+str(value) +',' )
                #print(self.addField(key,value))
                self.addField(key,value,reset_)

    def flatResult(self,reset_=False):
        self.flat_stat=self.stat_default.copy()
        self.flatJson(self.stats,reset_)
        
    def setStatData2(self):
        
        self.stat_data = [{"measurement":self.measurement,
                           "tags":{'model':self.model,
                                   'ip':self.host,
                                   'mc_type':'Asic miner'},
                                   
                          }
        ]
        self.stat_data[0]['fields'] = self.flat_stat
    
    def modelShortName(self):
        if('S19j' in self.model):
            return 'S19JPro'
        elif('L3+ Hiveon' in self.model):
            return 'L3'
        else:
            return 'L3'
        
    def initConnect(self):
        if(self.command("stats")): 
            self.stats = self.cmd_result
            self.model=self.stats["STATS"][0]["Type"]
            self.measurement='Asic_'+self.modelShortName()+'_' + api_version
            self.flatResult(True)
            self.stat_default= self.flat_stat.copy()
            #print(self.stat_default)
            
    def getMinerStats(self):
        self.resetStat()
        self.cmd_result={}
        self.flat_stat = self.stat_default.copy()
        if(self.command("stats")):
            self.stats = self.cmd_result
            self.flatResult()
            self.setStatData2()
        else:
            print('connect failed : reset STATS to default')
            #self.flat_stat = self.stat_default.copy()
            #print(self.stat_default)
            self.setStatData2()
        
    def command(self, command, arg=None):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5.0)
        self.connected = False
        self.cmd_result={}
        try:
            sock.connect((self.host, self.port))
            payload = {"command": command}
            if arg is not None:
                # Parameter must be converted to basestring (no int)
                payload.update({'parameter': unicode(arg)})

            sock.send(bytes(json.dumps(payload),'UTF-8'))
            received = self._receive(sock)
            self.cmd_result = json.loads(re.sub('}{', '},{',received[:-1]))
            self.connected = True
        except socket.timeout as err:
            print("time out"+ self.host)
            return False
        except OSError as e:
            #self.resetStat()  
            print(self.host)
            print (e)
        except Exception as e:
            print (self.host, e)
            return False
        finally:
            #print("finally "+self.host)
            if self.connected :
                try:
                    sock.shutdown(socket.SHUT_RDWR)
                    sock.close()
                except Exception as e:
                    print(self.host)
                    print(e)
                    return False
        
        return self.connected

    def _receive(self, sock, size=4096):
        msg = ''
        while 1:
            chunk = sock.recv(size).decode('UTF-8')
            if chunk:
                msg += chunk
            else:
                break
        return msg

    def __getattr__(self, attr):
        def out(arg=None):
            return self.command(attr, arg)
        return out

In [ ]:
# networkAddr = '192.168.1.'   , start_addr = 1  , to_addr = 254     
def generateIp(networkAddr, start_addr, to_addr):
    ipList = []    
    for i in range(start_addr,to_addr):
        ipList.append(networkAddr + str(i))
    
    return ipList

In [ ]:
def getMiners(ip_list):
    asic_list=[]
    for asic_ip in ip_list:
        miner = CgminerStats(asic_ip)
        miner.initConnect()
        if (miner.connected == True):
            asic_list.append(miner)
            #print (miner.description)
    return asic_list 
    
    

In [ ]:
def updateMinerStat(miner_list):
    for miner in miner_list:
        miner.getMinerStats()
        
    

In [ ]:
def insertInfluxDb(miner_list):
    try:
        with InfluxDBClient(url=url, token=token, org=org) as client:
                write_api = client.write_api(write_options=SYNCHRONOUS)               
                for miner in miner_list:
                    write_api.write(bucket, org, miner.stat_data)

                client.close()
    except Exception as e:
        print(e)
            

In [ ]:
def main():
    #iplist = generateIp('192.168.1.',1,254)
    miners = getMiners(iplist)
    print(len(miners))
    i=0
    while 1:
        updateMinerStat(miners)        
        insertInfluxDb(miners)
        i=i+1
        print (i)
        time.sleep(60)
    return miners


In [ ]:
main()

In [ ]:
#!ipython nbconvert --to script AsicMinerStat2Influxdb_3.ipynb